- https://github.com/ksopyla/awesome-nlp-polish
- Sentiment: https://pypi.org/project/sentimentpl/
- Auto correct: https://github.com/filyp/autocorrect
- other: https://github.com/sdadas/polish-nlp-resources
- papers: https://homados.ipipan.waw.pl/?page_id=93

In [17]:
# !pip install pandas tqdm pandarallel parallelbar spacy sentimentpl autocorrect --quiet
# !pip install sacremoses --quiet

In [2]:
# !python -m spacy download pl_core_news_lg

In [20]:
import pandas as pd
import numpy as np

from tqdm import tqdm

# progress bar
tqdm.pandas()

# from pandarallel import pandarallel
# pandarallel.initialize(nb_workers=7,progress_bar=True)

# from parallelbar import progress_map

# import spacy
# from sentimentpl.models import SentimentPLModel
# from autocorrect import Speller

# nlp_core = spacy.load("pl_core_news_lg")
# model = SentimentPLModel(from_pretrained='latest')
# spell = Speller('pl')

from helpers.utils import *

# CZ data

In [22]:
df_pl_cz = pd.read_csv('../datasets/demagog_nlp_cz/converted-exp-PL.tsv', sep='\t')

df_pl_cz['text_clean'] = df_pl_cz['statementText'].apply(lambda x: clean_przyp(x))

# tasks = df_pl_cz['text_clean'].values.tolist()
# result = df_pl_cz['text_clean'].progress_map(extract_features)
# result = progress_map(extract_features, tasks, n_cpu=7, chunk_size=1, core_progress=True)
df_pl_cz['raw_dict'] = df_pl_cz['text_clean'].progress_map(extract_features)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2835/2835 [11:55<00:00,  3.96it/s]


In [23]:
# df_pl_cz['raw_dict'] = result 

df_clean = (
    df_pl_cz.join( 
        df_pl_cz['raw_dict']
        .apply(pd.Series)
    ).drop('raw_dict', axis=1)
)

df_clean['TEXT_POS'] = df_clean['TEXT_POS'].str.join(" ")
df_clean['TEXT_WORD'] = df_clean['TEXT_WORD'].str.join(" ")

In [26]:
df_clean.to_parquet('../datasets/ready2use/fake_news_features_cz.parquet')

# Demagog

In [27]:
df = pd.read_csv('../datasets/scrapped/demagog.csv', sep=';')

df = df.dropna()
df = df[df['text'].str.len() > 0 ]

df['text_clean'] = df['text'].apply(lambda x: clean_przyp(x))

df = df[df['text_clean'].str.len() > 1 ]

df['raw_dict'] = df['text_clean'].progress_map(extract_features)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4917/4917 [17:58<00:00,  4.56it/s]


In [28]:
# df['raw_dict'] = result 

df_clean = (
    df.join( 
        df['raw_dict']
        .apply(pd.Series)
#         .rename(columns={'sentiment_lemm' : 'uniq_lemm'}) 
    ).drop('raw_dict', axis=1)
)

df_clean['TEXT_POS'] = df_clean['TEXT_POS'].str.join(" ")

In [29]:
df_clean.to_parquet('../datasets/scrapped/demagog_features.parquet')

# OKO.press

In [30]:
df_oko_raw = pd.read_csv('../datasets/oko.press/query_result.tsv', sep='\t')

print(df_oko_raw['Id wypowiedzi'].unique().shape)

df_oko = pd.merge(
    df_oko_raw.pivot(index=['Id wypowiedzi'], columns='Nazwa pola danych', values='Wartość pola danych').reset_index(),
    df_oko_raw[~df_oko_raw['Autor Wypowiedzi'].isin(['Link do hasła', 'Nazwa słupka', 'Wesprzyj nas'])] \
        [['Id wypowiedzi', 'Autor Wypowiedzi']].dropna(),
    on='Id wypowiedzi',
    how='left'
)

print(df_oko.shape)

(2869,)
(2869, 9)


In [31]:
df_oko_fin = df_oko[['sub_napis', 'sub_stan_zegara', 'Autor Wypowiedzi']]

df_oko_fin['text_clean'] = df_oko_fin['sub_napis'].progress_apply(lambda x: clean_przyp(x))

# tasks = df_oko_fin['text_clean'].values.tolist()
# result = progress_map(extract_features, tasks, n_cpu=7, chunk_size=1, core_progress=True)

df_oko_fin['raw_dict'] = df_oko_fin['text_clean'].progress_map(extract_features)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2869/2869 [00:00<00:00, 188225.71it/s]
/tmp/ipykernel_1100/518407814.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oko_fin['text_clean'] = df_oko_fin['sub_napis'].progress_apply(lambda x: clean_przyp(x))
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2869/2869 [13:55<00:00,  3.43it/s]
/tmp/ipykernel_1100/518407814.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [32]:
# df_oko_fin['raw_dict'] = result

df_oko_clean = df_oko_fin.join( df_oko_fin['raw_dict'].apply(pd.Series) ).drop('raw_dict', axis=1)

df_oko_clean['TEXT_POS'] = df_oko_clean['TEXT_POS'].str.join(" ")

In [33]:
df_oko_clean.to_parquet('../datasets/oko.press/okopress_features.parquet')

## Join data

In [35]:
df_dem = pd.read_parquet('../datasets/scrapped/demagog_features.parquet')
df_oko = pd.read_parquet('../datasets/oko.press/okopress_features.parquet')

df = pd.concat(
    [
        df_dem[
            ['assestment', 'author', 'text_clean', 'TEXT_POS',
             'sentiment_all', 'sentiment_avg', 'uniq_words', 'uniq_lemm', 
             'err', 'net', 
             # 'ADJ', 'ADV', 'NOUN', 
             'words_start_upper', 'words_full_upper',
             'exclamation_marks', 'question_marks', 
             'upper_letters', 'chars']
        ],
        (df_oko[
            ['sub_stan_zegara', 'Autor Wypowiedzi', 'text_clean', 'TEXT_POS',
             'sentiment_all', 'sentiment_avg', 'uniq_words', 'uniq_lemm', 
             'err', 'net', 
             # 'ADJ', 'ADV', 'NOUN', 
             'words_start_upper', 'words_full_upper',
             'exclamation_marks', 'question_marks', 
             'upper_letters', 'chars']
         ]
         .rename(columns={'sub_stan_zegara':'assestment', 'Autor Wypowiedzi':'author'}))
    ],
    ignore_index = True
)

In [36]:
df.to_parquet('../datasets/ready2use/fake_news_features_combined.parquet')